In [1]:
import numpy as np
import struct
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
from array import array
from os.path  import join
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
def initialize_parameters():


    W1 = np.random.randn(36,784)*0.01
    b1 = np.zeros((36,1))
    W2 = np.random.randn(10,36)*0.01
    b2 = np.zeros((10,1))

    return W1,b1,W2,b2

In [3]:
x,y = mnist['data'],mnist['target']

x_train,x_test = x[:60000], x[60000:]
y_train,y_test = y[:60000], y[60000:]

x_train = np.array(x_train,dtype='int32')
x_test = np.array(x_test,dtype = 'int32')
y_train = np.array(y_train,dtype = 'int32')
y_test = np.array(y_test,dtype='int32')

In [4]:


y_train = y_train.reshape(1,60000)
y_test = y_test.reshape(1,10000)
x_train = x_train.reshape(60000,-1).T
x_test = x_test.reshape(10000,-1).T

x_train = x_train/255
x_test = x_test/255



In [5]:
def relu(x):
  y = np.maximum(0,x)
  return y

def softmax(x):
  y = np.exp(x) / np.sum(np.exp(x), axis = 0, keepdims = True)
  return y

def forward(W1, b1, W2, b2,X):
  Z1 = np.dot(W1,X) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2,A1) + b2
  A2 = softmax(Z2)
  return Z1, A1, Z2, A2



In [6]:
def compute_cost(A2, Y):
  m = Y.size
  cost = -1/m * np.sum(np.dot(Y,np.log(A2).T) + np.dot((1-Y),np.log(1-A2).T))
  return cost

In [7]:
def relu_backward(z):
  return z >= 0

def one_hot(Y):
  encoded_array = np.zeros((Y.size, Y.max() + 1),dtype = int)
  encoded_array[np.arange(Y.size), Y] = 1
  encoded_array = encoded_array.T
  return encoded_array

In [8]:


def backward(Z1, A1, Z2, A2, W2, X, Y):
  m = Y.size
  one_hot_Y = one_hot(Y)
  dZ2 = A2 - one_hot_Y
  dW2 = 1/m * np.dot(dZ2,A1.T)
  db2 = 1/m * np.sum(dZ2, axis =1, keepdims = True)
  dZ1 = np.dot(W2.T, dZ2) * relu_backward(Z1)
  dW1 = 1/m * np.dot(dZ1,X.T)
  db1 = 1/m * np.sum(dZ1,axis = 1, keepdims = True)
  return dW1, db1, dW2, db2




In [9]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, Learning_rate):

  W1 = W1 - Learning_rate*dW1
  W2 = W2 - Learning_rate*dW2
  b1 = b1 - Learning_rate*db1
  b2 = b2 - Learning_rate*db2

  return W1, b1, W2, b2

def get_predictions(A2):
  return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
  print(predictions, Y)
  return np.sum(predictions == Y)/ Y.size

In [12]:
def model(X, Y, iterations, learning_Rate):
  W1, b1, W2, b2 = initialize_parameters()

  for i in range(0, iterations):
    costs = []
    Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
    cost = compute_cost(A2, Y)
    dW1, db1, dW2, db2 = backward(Z1, A1, Z2, A2, W2, X, Y)
    W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_Rate)
    if i%100 == 0:
      print("Cost after " + str(i) + "th Iteration is " + str(cost))
    if i % 100 == 0 or i == iterations:
      print("Iterations: " + str(i))
      print("Accuracy  :" + str(get_accuracy(get_predictions(A2), Y)))
  return cost,W1,b1,W2,b2

In [13]:
cost,W1,b1,W2,b2 = model(x_train, y_train, 3000, 0.1)


Cost after 0th Iteration is 98.91697024318387
Iterations: 0
[3 6 3 ... 3 6 3] [[5 0 4 ... 5 6 8]]
Accuracy  :0.1219
Cost after 100th Iteration is 123.71416017286083
Iterations: 100
[3 0 4 ... 7 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.7050833333333333
Cost after 200th Iteration is 218.62763795300742
Iterations: 200
[3 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.8492833333333333
Cost after 300th Iteration is 265.534354734101
Iterations: 300
[3 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.8803333333333333
Cost after 400th Iteration is 291.18387966497824
Iterations: 400
[3 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.8934666666666666
Cost after 500th Iteration is 306.8504176014952
Iterations: 500
[3 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.9002833333333333
Cost after 600th Iteration is 317.26802776995436
Iterations: 600
[3 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]]
Accuracy  :0.9054
Cost after 700th Iteration is 324.73080630236683
Iterations: 700
[5 0 4 ... 5 6 8] [[5 0 4 ... 5 6 8]

In [14]:
Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, x_test)
predictions = get_predictions(A2)
print("Accuracy  :" + str(get_accuracy(predictions, y_test)))

[7 2 1 ... 4 5 6] [[7 2 1 ... 4 5 6]]
Accuracy  :0.9433
